In [2]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')

from datetime import timedelta

In [3]:
from config_paper import key_id, secret_key
from cross_signal import *

In [4]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(key_id, secret_key,base_url)

In [5]:
today = datetime.datetime.today().date() # or datetime.now to use local timezone
daystart=pd.Timestamp(year=today.year, month=today.month, day=today.day, hour=0,tz='America/New_York').isoformat()

In [6]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        dataframe['ultosc'] = ta.ULTOSC(dataframe['high'],dataframe['low'],dataframe['close'])
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=20, price='close')
        dataframe['adx'] = tab.ADX(dataframe)

        # required for graphing

        return dataframe

In [7]:
account = api.get_account()
print(account.status)
account.buying_power

ACTIVE


'141284'

In [8]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [9]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.01

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)
trade=True

In [1]:
trade=True
while trade==True:
    #asset Manager
    currenthour=time.localtime().tm_hour
    currentminute=time.localtime().tm_min
    currenttime=currenthour*100+currentminute
    
    
    assetlist=api.list_positions()

    for asset in assetlist:
            #if asset is in list of positions AND is more than 2.5% loss
            # sell and turn that symbol to null

        if float(asset.unrealized_intraday_plpc) > .003:
            symbol=asset.symbol
            try:
                returned_data = api.get_barset(symbol,barTimeframe,limit=150,start=daystart).df
                #api.get_barset(symbol,barTimeframe,limit=150).df


                # Calculated trading indicators
                #populate exponential moving averages
                data=populateindicators(returned_data[symbol])

                if data['plusdi'][-1] < 55 and data['minusdi'][-1] > 45:

                    openPosition = api.get_position(asset.symbol)

                    returned = api.submit_order(symbol,int(openPosition.qty),"sell","market","day") # Market order to fully close position
                    print('Sell',returned)
            except:
                print(symbol)

    #get current time and shut off trading
 
    if currenthour >= 12: 
        trade=False
        print('Good work, Turning off')
        api.close_all_positions
    #interval number of seconds to wait
    time.sleep(5)
    
    

NameError: name 'time' is not defined

In [15]:
api.close_all_positions()

[Order({   'body': {   'asset_class': 'us_equity',
                 'asset_id': '48cbc81a-1c23-4356-92b0-207871c6f6a3',
                 'canceled_at': None,
                 'client_order_id': 'a6ab5f1a-8980-46aa-9b2b-c6ec2cfc0cb2',
                 'created_at': '2020-04-29T18:47:07.627234Z',
                 'expired_at': None,
                 'extended_hours': False,
                 'failed_at': None,
                 'filled_at': None,
                 'filled_avg_price': None,
                 'filled_qty': '0',
                 'id': 'c7e8d617-1e98-4d07-a625-4622433f4713',
                 'legs': None,
                 'limit_price': None,
                 'order_class': '',
                 'order_type': 'market',
                 'qty': '73',
                 'replaced_at': None,
                 'replaced_by': None,
                 'replaces': None,
                 'side': 'sell',
                 'status': 'accepted',
                 'stop_price': None,
               